Reproduce train-val-test split on which the model 'CNN_128x128x2_3layers_3cl' was trained, and save the results in text files. TOWRITE

Save list of str in a file (TOCHECK):

with fopen('train.txt', 'w') as fname:
    print('\n'.split(files_train), file=fname)


After this operation:
    copy the specified files using names in text file;
    call the function for prediction


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

# import cv2

# from sklearn.model_selection import train_test_split
# from sklearn import metrics


# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
# from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback

# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.utils import to_categorical
# from timeit import default_timer as timer


# imports for loading and plotting data.
# from load_images_limit_masks_v2 import list_img, load_data_masks

# from load_images_limit_masks_v3 import list_img, load_data_masks
from load_images_limit_masks_v4 import list_img, load_data_masks

# from train_val_test_split import train_val_test_split
# most used functions in this script.

from sklearn.model_selection import train_test_split

import time
from functools import partial

image_path = '../data'

In [2]:
# Hyperparamètres (réduction de l'image)

IMG_HEIGHT = 128
IMG_WIDTH = 128
# L'architecture du CNN peut aussi être déplacée ici.

In [3]:
def cond2folder_img(condition):
    return os.path.join(image_path, condition, "images")

def cond2folder_masks(condition):
    return os.path.join(image_path, condition, "masks")

Charger les images: 2500 COVID, (1250 Pneumo + 1250 Opacity), 2500 normaux choisies aléatoirement 

In [4]:
lst_conditions = ['COVID', 'Viral Pneumonia', 'Lung_Opacity', 'Normal']
lst_dirs_img = list(map(cond2folder_img, lst_conditions))
print(lst_dirs_img)

# 
lst_dirs_masks = list(map(cond2folder_masks, lst_conditions))
print(lst_dirs_masks)


lst_labels = [2, 1, 1, 0]

print("labels:", lst_labels)

['../data/COVID/images', '../data/Viral Pneumonia/images', '../data/Lung_Opacity/images', '../data/Normal/images']
['../data/COVID/masks', '../data/Viral Pneumonia/masks', '../data/Lung_Opacity/masks', '../data/Normal/masks']
labels: [2, 1, 1, 0]


In [5]:
df_excluded = pd.read_csv("outliers_list.csv", names=["fname"])

print(df_excluded.info())
df_excluded.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   fname   106 non-null    object
dtypes: object(1)
memory usage: 976.0+ bytes
None


,fname
0,COVID-135
1,COVID-509
2,COVID-604
3,COVID-653
4,COVID-686


In [6]:
# determine the number of images to load


lst_nums_images = list(map(lambda path_dir: len(list_img(path_dir, excluded=df_excluded)), lst_dirs_img ))
print("Total number of images for each condition:", lst_nums_images)
# (before removing images) [3616, 1345, 6012, 10192]
# [3513, 1344, 6012, 10190]


Total number of images for each condition: [3513, 1344, 6012, 10190]


In [7]:
# Number of images to extract:
num_img_COV = 2500

balanced_lst_nums_images = [num_img_COV, num_img_COV//2, num_img_COV//2, num_img_COV]

print(balanced_lst_nums_images)
# [2500, 1250, 1250, 2500]



[2500, 1250, 1250, 2500]


Extract images. Adapted to loading masks as well.

Adapted to getting file names as well.

In [8]:
start = time.time()
# The function replaces these instructions:
"""
data_img, labels = load_data(lst_dirs_img,
                        lst_labels,
                        new_size=(IMG_WIDTH, IMG_HEIGHT),
                        limits=balanced_lst_nums_images,
                        random_state=42)

data_masks, __ = load_data(lst_dirs_masks,
                        lst_labels,
                        new_size=(IMG_WIDTH, IMG_HEIGHT),
                        limits=balanced_lst_nums_images,
                        random_state=42)

data = np.concatenate((data_img, data_masks), axis=3)
"""

"""
# instruction for training the model,
# throws out the file names
data, labels, __ = load_data_masks([lst_dirs_img, lst_dirs_masks],
                        lst_labels,
                        new_size=(IMG_WIDTH, IMG_HEIGHT),
                        limits=balanced_lst_nums_images,
                        random_state=42)
"""
data, labels, lst_lst_files = load_data_masks([lst_dirs_img, lst_dirs_masks],
                        lst_labels,
                        new_size=(IMG_WIDTH, IMG_HEIGHT),
                        excluded=df_excluded,
                        limits=balanced_lst_nums_images,
                        random_state=42)


done = time.time()

elapsed = done - start
print(f'Loading data : {elapsed:.2f} sec \n\n')
# 11 sec

print(type(data))  # ndarray
print(data.shape)  # (7500, 128, 128, 2)
print(type(labels))  # ndarray
print(labels.shape)  # (7500,)

print('--- files: ------')
print(type(lst_lst_files))  # list
print(len(lst_lst_files))  # 2

print(type(lst_lst_files[0]))  # list
print(len(lst_lst_files[0]))  #  7500

print(type(lst_lst_files[0][0]))  # np.str_
print(len(lst_lst_files[0][0]))  # 13
print(lst_lst_files[0][0])  # COVID-733.png

print(type(lst_lst_files[1][0]))  # np.str_
print(len(lst_lst_files[1][0]))  # 13
print(lst_lst_files[1][0])  # COVID-733.png

Loading data : 10.87 sec 


<class 'numpy.ndarray'>
(7500, 128, 128, 2)
<class 'numpy.ndarray'>
(7500,)
--- files: ------
<class 'list'>
2
<class 'list'>
7500
<class 'numpy.str_'>
13
COVID-733.png
<class 'numpy.str_'>
13
COVID-733.png


In [9]:
# Check the distribution of labels.
lbl_unique, lbl_counts = np.unique(labels, return_counts=True)

print(lbl_unique)  # ['0 1 2]
print('--------')
print(lbl_counts)  # [2500 2500 2500]

# As expected, the classes are balanced.


[0 1 2]
--------
[2500 2500 2500]


# Train-val-test split

Adapted to getting file names as well.

I should call 'train_val_test_split' with the indices in range(0, data.shape[0])
as 2nd argument.

In [10]:
def index_lst_by_lst(full_lst: list, indices: list):
    """
    See: https://stackoverflow.com/a/22412533

    Args:
        full_lst (list): list with all elements 
        indices (list of int): indices
    """
    return [full_lst[index] for index in indices]

# example
print(index_lst_by_lst(['0', '1', '2', '3', '4', '5', '6'], [0,2,4,5]))

['0', '2', '4', '5']


In [11]:
"""
# this cannot be immediately be adapted to 4 vectors
# (data, labels, lst_lst_files[0], lst_lst_files[1])
# for this possibility has not been provided in the function train_val_test_split .

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(data,
                                                                labels,
                                                                random_state=42)

# ->
"""
ind_train_val, ind_test, y_train_val, y_test = train_test_split(
        range(0, data.shape[0]),
        labels, # added
        stratify=labels, # added
        test_size=0.1,
        random_state=42)

ind_train, ind_val, y_train, y_val = train_test_split(
        ind_train_val,
        y_train_val, # added
        stratify=y_train_val, # added
        test_size=0.2,
        random_state=42)

print("The training indices are of type:", type(ind_train))
print("The 1st values are:", ind_train[:5])  # [5649, 2514, 6263, 6082, 1170]

X_train = data.take(ind_train, axis=0)
X_val = data.take(ind_val, axis=0)
X_test = data.take(ind_test, axis=0)

# y_train = labels[ind_train]
# y_val = labels[ind_val]
# y_test = labels[ind_test]

"""
img_train = lst_lst_files[0][ind_train]
img_val = lst_lst_files[0][ind_val]
img_test = lst_lst_files[0][ind_test]

masks_train = lst_lst_files[1][ind_train]
masks_val = lst_lst_files[1][ind_val]
masks_test = lst_lst_files[1][ind_test]
"""

lst_all_img = lst_lst_files[0]
lst_all_masks = lst_lst_files[1]

img_train, img_val, img_test = tuple(map(partial(index_lst_by_lst, lst_all_img),
                                        [ind_train, ind_val, ind_test]))


masks_train, masks_val, masks_test = tuple(map(partial(index_lst_by_lst, lst_all_masks),
                                               [ind_train, ind_val, ind_test]))


# show the split



print(X_train.shape)   # (5400, 784)
print(X_val.shape)     # (1350, 784)
print(X_test.shape)    # (750, 784)

print(y_train.shape)   # (5400,)
print(y_val.shape)     # (1350,)
print(y_test.shape)    # (750,)

print(len(img_train))   # 5400
print(len(masks_train)) # 5400
print(img_train[:5])
# ['Normal-1439.png', 'Viral Pneumonia-1249.png', 'Normal-10035.png', 'Normal-4287.png', 'COVID-1535.png']
# (before adding stratification) ->
# ['COVID-2242.png', 'Normal-10021.png', 'COVID-1998.png', 'Viral Pneumonia-1303.png', 'COVID-3125.png']

print(masks_train[:5])

The training indices are of type: <class 'list'>
The 1st values are: [1535, 5761, 251, 3218, 1162]
(5400, 128, 128, 2)
(1350, 128, 128, 2)
(750, 128, 128, 2)
(5400,)
(1350,)
(750,)
5400
5400
['COVID-2242.png', 'Normal-10021.png', 'COVID-1998.png', 'Viral Pneumonia-1303.png', 'COVID-3125.png']
['COVID-2242.png', 'Normal-10021.png', 'COVID-1998.png', 'Viral Pneumonia-1303.png', 'COVID-3125.png']


Save the file names and classes to CSV.


In [12]:
def fname2folder(fname: str):
    """
    file mame -> subfolder of the folder "data",
    where the file is located.

    Args:
        fname (str): file name.
    Returns:
        Name of the subfolder.
    """
    return fname.split('-')[0]

# example
print(fname2folder('COVID-42.png'))  # COVID

COVID


In [13]:
def fname2img_folder(fname: str):
    """
    file mame -> subfolder of the folder "data",
    where the image file is located.

    Args:
        fname (str): file name.
    Returns:
        Name of the subfolder.
    """
    return os.path.join(fname2folder(fname), "images")

def fname2mask_folder(fname: str):
    """
    file mame -> subfolder of the folder "data",
    where the mask file is located.

    Args:
        fname (str): file name.
    Returns:
        Name of the subfolder.
    """
    return os.path.join(fname2folder(fname), "masks")

In [14]:
# column names. Placed to variables
# for they are common for all 3 DataFrames. 

var_Img = 'Images'
var_masks = 'Masks'
var_num_class = 'num_class'
var_img_folder = 'img_folder'
var_mask_folder = 'mask_folder'

In [15]:
def fnames_y_to_df(img_train, masks_train, y_train):
    """
    Generates a DataFrame from filenames and label.

    Args:
        img_train (str):
        masks_train (str):
        y_train (int):
    """
    df_files_train = pd.DataFrame(data={var_Img: img_train, var_masks: masks_train, var_num_class: y_train})
    df_files_train[var_img_folder] = df_files_train[var_Img].apply(fname2img_folder)
    df_files_train[var_mask_folder] = df_files_train[var_masks].apply(fname2mask_folder)
    return df_files_train

In [16]:
df_files_train = fnames_y_to_df(img_train, masks_train, y_train)


print(df_files_train.info())  # (5400, 5)
df_files_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5400 entries, 0 to 5399
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Images       5400 non-null   object
 1   Masks        5400 non-null   object
 2   num_class    5400 non-null   int64 
 3   img_folder   5400 non-null   object
 4   mask_folder  5400 non-null   object
dtypes: int64(1), object(4)
memory usage: 211.1+ KB
None


,Images,Masks,num_class,img_folder,mask_folder
0,COVID-2242.png,COVID-2242.png,2,COVID/images,COVID/masks
1,Normal-10021.png,Normal-10021.png,0,Normal/images,Normal/masks
2,COVID-1998.png,COVID-1998.png,2,COVID/images,COVID/masks
3,Viral Pneumonia-1303.png,Viral Pneumonia-1303.png,1,Viral Pneumonia/images,Viral Pneumonia/masks
4,COVID-3125.png,COVID-3125.png,2,COVID/images,COVID/masks


In [17]:
# df_files_train.to_csv('../src/models/datasets1_2/img_train.csv')

In [18]:
# Check the images have been excluded indeed.
ser_fnames_train = df_files_train[var_Img].apply(lambda full_fname: full_fname[:-4])

ser_fnames_train.head()

0              COVID-2242
1            Normal-10021
2              COVID-1998
3    Viral Pneumonia-1303
4              COVID-3125
Name: Images, dtype: object

In [19]:
ser_should_be_excluded_fnames = ser_fnames_train.isin(df_excluded)
print(sum(ser_should_be_excluded_fnames))  # 0, OK

0


In [39]:
df_files_val = fnames_y_to_df(img_val, masks_val, y_val)


print(df_files_val.info())  # (5400, 5)
df_files_val.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Images       1350 non-null   object
 1   Masks        1350 non-null   object
 2   num_class    1350 non-null   int64 
 3   img_folder   1350 non-null   object
 4   mask_folder  1350 non-null   object
dtypes: int64(1), object(4)
memory usage: 52.9+ KB
None


,Images,Masks,num_class,img_folder,mask_folder
0,COVID-1842.png,COVID-1842.png,2,COVID/images,COVID/masks
1,COVID-3413.png,COVID-3413.png,2,COVID/images,COVID/masks
2,Viral Pneumonia-77.png,Viral Pneumonia-77.png,1,Viral Pneumonia/images,Viral Pneumonia/masks
3,Lung_Opacity-5556.png,Lung_Opacity-5556.png,1,Lung_Opacity/images,Lung_Opacity/masks
4,COVID-2980.png,COVID-2980.png,2,COVID/images,COVID/masks


In [40]:
# df_files_val.to_csv('../src/models/datasets1_2/img_val.csv')

In [41]:
df_files_test = fnames_y_to_df(img_test, masks_test, y_test)


print(df_files_test.info())  # (5400, 5)
df_files_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Images       750 non-null    object
 1   Masks        750 non-null    object
 2   num_class    750 non-null    int64 
 3   img_folder   750 non-null    object
 4   mask_folder  750 non-null    object
dtypes: int64(1), object(4)
memory usage: 29.4+ KB
None


,Images,Masks,num_class,img_folder,mask_folder
0,Normal-5305.png,Normal-5305.png,0,Normal/images,Normal/masks
1,Viral Pneumonia-748.png,Viral Pneumonia-748.png,1,Viral Pneumonia/images,Viral Pneumonia/masks
2,Normal-3718.png,Normal-3718.png,0,Normal/images,Normal/masks
3,Lung_Opacity-4849.png,Lung_Opacity-4849.png,1,Lung_Opacity/images,Lung_Opacity/masks
4,Normal-789.png,Normal-789.png,0,Normal/images,Normal/masks


In [42]:
# df_files_test.to_csv('../src/models/datasets1_2/img_test.csv')